# Step 1. Let's bomb your model!

This script bombs your model on our little red-teaming evaluation dataset and saves answers of your model into the file.

You can upload this file to our benchmark if you want to get metrics OR you can run the bench.py file to get results yourself.

## Preparing

You need to set up first things out - load your model.

Do it in custom way or use our supported.

### Loading supported API model

Create and place api_keys.json to the repo:
`this_repo_folder/config/api_keys.json`

api_keys must contain next structure:
```json
{
    "openai": {
        "key": "YOUR-OPENAI-KEY"
    },
    "langchain": {
        "key": "YOUR-LANGCHAIN-KEY"
    },
    "yandex": {
        "id": "YANDEX-ID",
        "key": "YANDEX-API-KEY",
        "folder_id": "YANDEX-FOLDER-ID"
    },
    "gigachat": {
        "client_id": "GIGACHAT-CLIENT-ID",
        "secret": "GIGACHAT-CLIENT-SECRET",
        "auth": "GIGACHAT-CLIENT-AUTH-CODE"
    },
    "vsegpt": {
        "base_url": "https://api.vsegpt.ru/v1",
        "key": "VSEGPT-API-KEY"
    }
}
```

INSTALL ALL DEPENDS FOR LANGCHAIN, YANDEXGPT, OPENAI, etc...

### Loading custom model

If you use your custom model, just provide it to the this "generate" function:

```python
def generate(system_prompt: str, user_prompt: str) -> str:

    # your function initialization, in example:
    # return model.generate(f"""system:

{system_prompt}

user:

{user_prompt}

assistant: """)
```

Otherwise, use our
```
import generate from benching
```

INSTALL ONLY pandas / other little things needed

In [ ]:
# TODO load benchmark data
# TODO generate answers on data
# TODO save results

# Step 2. Let's SCORE your model

Scoring your model outputs!

Ensure all dependencies is installed & you need to specify & load the judge model.

In [ ]:
!python this_repo_folder/benching/bench.py